In [ ]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent))
# sys.path

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate


In [ ]:
%load_ext autoreload
%autoreload 2
%aimport optiml.queries

## Initialization

In [ ]:
# Initialize Snowflake connection to Snowflake
from optiml.connection import SnowflakeConnConfig
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
# Initialize local environment
import os
cache_dir = os.path.expanduser('~/data/kiva')
# Initialize query library
from optiml.queries import SNFLKQuery
qlib = SNFLKQuery(connection, 'KIV', cache_dir)
sdate = '2022-10-01'
edate = '2022-10-31'
color_scheme=["red","blue","green","orange","purple","brown","pink","gray","olive","cyan","darkviolet","goldenrod","darkgreen","chocolate","lawngreen"]

# Total cost breakdown 

## Cost by usage category

In [ ]:
df = qlib.total_cost_breakdown_ts(sdate, edate).round(2)
df.head()

In [ ]:
df_by_usage_category = df.groupby("category_name").sum("numeric_only").reset_index()
df_by_usage_category.loc[len(df.index)] = ['Total', df['credits'].sum(), df['dollars'].sum()]
print(tabulate(df_by_usage_category, headers='keys', tablefmt='rounded_outline', showindex=False))

In [ ]:
## Remove the last row of totals for the plot
df_by_usage_category.reset_index(inplace=True)
df_by_usage_category.drop(columns=["index"], inplace=True)
df_by_usage_category = df_by_usage_category.drop(len(df_by_usage_category)-1) 


In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"},{"type": "pie"}]],
    subplot_titles=("Dollars", "Credits")
)

fig.add_trace(go.Pie(labels=df_by_usage_category['category_name'].tolist(), values=df_by_usage_category['dollars'].tolist(),name="Dollars", rotation=45,marker_colors=color_scheme),row=1,col=1)
fig.add_trace(go.Pie(labels=df_by_usage_category['category_name'].tolist(), values=df_by_usage_category['credits'].tolist(),name='Credits', rotation=45,marker_colors=color_scheme),row=1,col=2)

fig.update_layout(
    title={
        'text': "Breakdown of total cost by usage category",
        'y':0.1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'bottom'})
fig.show()

## Cost by usage category timeseries

In [ ]:
df_by_category_ts = df.groupby(['category_name','start_time','end_time']).sum('numeric_only').reset_index()
fig = px.area(df_by_category_ts, x="start_time", y="credits", color="category_name",color_discrete_sequence=color_scheme)
fig.show()

## Cost by user

In [ ]:
df_by_user = df.groupby(['user_name']).sum('numeric_only').reset_index()
print(tabulate(df_by_user, headers='keys', tablefmt='rounded_outline', showindex=False))

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"},{"type": "pie"}]],
    subplot_titles=("Dollars", "Credits")
)

fig.add_trace(go.Pie(labels=df_by_user['user_name'].tolist(), values=df_by_user['dollars'].tolist(),name="Dollars", rotation=45,marker_colors=color_scheme),row=1,col=1)
fig.add_trace(go.Pie(labels=df_by_user['user_name'].tolist(), values=df_by_user['credits'].tolist(),name='Credits', rotation=45,marker_colors=color_scheme),row=1,col=2)

fig.update_layout(
    title={
        'text': "Breakdown of total cost by user",
        'y':0.1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'bottom'})
fig.show()

In [ ]:
df_by_user_ts = df.groupby(['user_name','start_time','end_time']).sum('numeric_only').reset_index()
fig = px.area(df_by_user_ts, x="start_time", y="credits", color="user_name",color_discrete_sequence=color_scheme)
fig.show()

## Cost by warehouse

In [ ]:
df = qlib.cost_by_wh_ts(sdate, edate).round(2)
df.head()
# df.loc[len(df.index)] = ['Total', df['credits'].sum(), df['dollars'].sum()]
# print(tabulate(df, headers='keys', tablefmt='rounded_outline', showindex=False))

In [ ]:
df = df.drop(len(df)-1) ## Remove the last row of totals for the plot

fig = make_subplots(
    rows=1, cols=1,
    specs=[[{"type": "pie"}]],
)

fig.add_trace(go.Pie(labels=df['warehouse_name'].tolist(), values=df['dollars'].tolist(),name='dollars',marker_colors=color_scheme),row=1,col=1)

fig.update_layout(
    title={
        'text': "Breakdown of total cost by warehouse",
        'y':0.1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [ ]:
df_by_warehouse_ts=df.groupby(['warehouse_name','start_time','end_time']).sum('numeric_only').reset_index()
fig = px.area(df_by_user_ts, x="start_time", y="credits", color="user_name",color_discrete_sequence=color_scheme)
fig.show()


## Analysis
- Sum of the compute across all the warehouses is 1255.47 credits which is consistent with total cost of compute above. Sanity check to ensure we are not missing any compute or warehouse

## Breakdown of cost by partner tools

In [ ]:
df = qlib.cost_by_partner_tool_ts(sdate, edate).round(2)
df = df.drop(len(df)-1).groupby('client_application_name').sum('numeric_only').reset_index()
df.loc[len(df.index)] = ['Total', df['approximate_credits_used'].sum()]
print(tabulate(df, headers='keys', tablefmt='rounded_outline', showindex=False))

In [ ]:
df = df.drop(len(df)-1) ## Remove the last row of totals for the plot
fig = make_subplots(
    rows=1, cols=1,
    specs=[[{"type": "pie"}]],
)

fig.add_trace(go.Pie(labels=df_client_app['client_application_name'].tolist(), values=df_client_app['approximate_credits_used'].tolist(),name='credits'),row=1,col=1)

fig.update_layout(
    title={
        'text': "Breakdown of credits used by Client Apps",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

## Warehouse configuration

In [ ]:
df = qlib.warehouse_config()
print(tabulate(df, headers='keys', tablefmt='rounded_outline', showindex=False))

In [ ]:
df.columns

In [ ]:
df.name

## Analysis
- auto suspend times here vary from 60 min to 3600 min - these seem very large assuming auto resuming does not take very long
- There are no resource monitors being used - this should be corrected

## Cost of queries

In [ ]:
df = qlib.n_expensive_queries(sdate, edate, 10)
df

In [ ]:
df['query_text'][0] == df['query_text'][1]

In [ ]:
df['query_text'][0]

In [ ]:
df['query_text'][1]

In [ ]:
df = qlib.cost_by_wh_ts()
df.warehouse_name.unique()

In [ ]:
df=qlib.idle_users(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.users_full_table_scans(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.heavy_users(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.users_never_logged_in(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.users_never_logged_in(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.idle_roles(start_date="2022-01-01", end_date="2022-02-02")
df.head()


In [ ]:
df=qlib.failed_tasks(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.long_running_tasks(start_date="2022-01-01", end_date="2022-02-02")
df.head()